# Helper functions for Tensorflow Fold



In [ ]:
length = td.Length()

embedded =  (td.InputTransform(lambda s: [index(x) for x in s]) >> 
             td.Map(td.Scalar(tf.int32) >> 
             td.Function(td.Embedding(vsize, EMBEDDING_SIZE))))

onehot = (td.InputTransform(lambda s: [index(x) for x in s]) >>
          td.Map(td.Scalar(tf.int32) >>
          td.Function(lambda indices: tf.one_hot(indices, depth=vsize)) >>
          td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))))

decode_onehot = td.InputTransform(lambda s: [char(np.argmax(np.squeeze(x))) for x in s])

print(decode_onehot.eval(onehot.eval("malú")))
#embedded.eval("kacsa")

In [ ]:
def conv1d_on_sequence(x, scope, kernel_size=3, input_channels=72, output_channels=72):
    with tf.variable_scope(scope) as sc:
        filters = tf.get_variable("conv_filter", [kernel_size] +  [input_channels, output_channels] , initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias = tf.get_variable("conv_bias",  output_channels, initializer=tf.constant_initializer(0.05, dtype=tf.float32))
        conv = tf.nn.conv1d(x, filters=filters, stride=1, padding='VALID')
        return tf.nn.relu(tf.add(conv, bias))
    
def SeqToTuple(T, N):
    return (td.InputTransform(lambda x: tuple(x))
            .set_input_type(td.SequenceType(T))
            .set_output_type(td.Tuple(*([T] * N))))   

In [ ]:
def bidirectional_conv_LSTM():
    convlstm = Conv1DLSTMCell(input_shape=[vsize,1], output_channels=8, kernel_shape=[5])
    conv_lstm_cell_1d = td.ScopedLayer(convlstm)

    bidir_conv_lstm = td.Composition()
    with bidir_conv_lstm.scope():
        data = td.Record((td.Map(
                                td.Vector(vsize) >>
                                td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))),
                          td.Map(
                                td.Vector(vsize) >>
                                td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))))).reads(bidir_conv_lstm.input)

        forward = td.Identity().reads(data[0])
        backward = td.Identity().reads(data[1])

        forw = (td.RNN(conv_lstm_cell_1d) >>
                td.GetItem(1) >>
                td.GetItem(0) >>
                td.Function(lambda rnn_outs: tf.contrib.layers.flatten(rnn_outs))).reads(forward)

        backw = (td.RNN(conv_lstm_cell_1d) >>
                 td.GetItem(1) >>
                 td.GetItem(0) >>
                 td.Function(lambda rnn_outs: tf.contrib.layers.flatten(rnn_outs))).reads(backward)

        rnn_outs = td.Concat().reads(forw,backw)
        bidir_conv_lstm.output.reads(rnn_outs)
    return bidir_conv_lstm >> td.FC(1)


def FCNN():
    return td.FC(50) >> td.FC(1)# >> td.Function(lambda xs: tf.squeeze(xs, axis=1))

#bidir = bidirectional_conv_LSTM()
#fc = FCNN()
#blk = bidir# >> fc 
#blk.eval([a,a])

In [ ]:
cell = td.ScopedLayer(tf.contrib.rnn.BasicLSTMCell(num_units=16), 'char_cell')
blk = td.Map(td.Vector(vsize) >> td.Function(lambda x: tf.reshape(x, [-1,vsize]))) >> td.RNN(cell) >> td.GetItem(1) >> td.GetItem(0) >> td.FC(1)#>> td.Function(lambda xs: tf.squeeze(xs, axis=1))

In [ ]:
class reader():
    def __init__(self, folder, mode="bidirectional", qsize=10000, start=True):
        self.data_dir = folder
        self.qsize= qsize
        self.pool = dict()
        self.data = dict()
        self.datasets = ["train", "test", "validation"]
        if mode == "bidirectional":
            convert_fn = self.bidirectional_sentence_reader
        else:
            convert_fn = self.sentence_reader
            
        for dataset in self.datasets:
            self.data[dataset] = Queue(qsize)
            self.pool[dataset] = Process(target=convert_fn, args=(self.data[dataset], self.data_dir+dataset))
        if start == True:
            self.start()
            
    def sentence_reader(self, queue, file):
        """
        read sentences from the data format setence: word\tword\n.....\t\n
        """
        while True:
            with open(file) as f:
                while True:
                    try:
                        sentence = []
                        while True:
                            line = f.readline()[:-1].split('\t')
                            if line[0] != "":
                                sentence.append(line)
                            else:
                                break
                        sent = " ".join([word[0] for word in sentence])
                        segmented = " ".join([word[1].replace(" ","|") for word in sentence])
                        tags = []
                        last_char = "_"
                        for char in segmented:
                            if char != "|":
                                tags.append(0 if last_char!="|" else 1)
                            last_char = char
                        queue.put((sent, tags))
                    except e:
                        print(e)
                        
    def bidirectional_sentence_reader(self, queue, file):
        """
        read sentences from the data format setence: word\tword\n.....\t\n
        """
        while True:
            with open(file) as f:
                while True:
                    try:
                        sentence = []
                        while True:
                            line = f.readline()[:-1].split('\t')
                            if line[0] != "":
                                sentence.append(line)
                            else:
                                break
                        sent = " ".join([word[0] for word in sentence])
                        segmented = " ".join([word[1].replace(" ","|") for word in sentence])
                        tags = []
                        last_char = "_"
                        for char in segmented:
                            if char != "|":
                                tags.append(0 if last_char!="|" else 1)
                            last_char = char
                        for i in range(1, len(sent)-1):
                            forward, backward = sent[:i], sent[i:][::-1]
                            queue.put(([self.onehot(forward), self.onehot(backward)], tags[i]))
                    except:
                        print("err")
                        
                        
    def start(self):
        for dataset in self.datasets:
            self.pool[dataset].start()
            
    def stop(self):
        for dataset in self.datasets:
            self.pool[dataset].terminate()
            
    def get(self,dataset):
        if dataset in self.datasets:
            return self.data[dataset].get()
        else:
            raise KeyError
            
    def onehot(self, string):
        onehot = np.zeros([len(string),vsize])
        onehot[np.arange(len(string)), np.array([index(char) for char in string])]=1
        return [onehot[i,:] for i in range(len(onehot))]
            
#store = reader("/home/moon/data/", start=False)

